In [ ]:
import numpy as np
import time
import os
import cv2
import numba
import math
import threading
from queue import Queue
from sklearn.cluster import KMeans
import joblib

In [ ]:
################################  HONV  #####################################

In [ ]:
import HONV_Library as HONV

In [ ]:
################################  Read Images  #####################################

In [ ]:
# this function is for image reading,the input is directory name
def read_directory(directory_name):
    array_of_img = [] # this if for store all of the image data
    # this loop is for read each image in this foder,directory_name is the foder name with images.
    for filename in os.listdir(r"./"+directory_name):
        #print(filename) #just for test
        #img is used to store the image data 
        img = cv2.imread(directory_name + "/" + filename, cv2.IMREAD_ANYDEPTH)
        array_of_img.append(img)
    return array_of_img

In [ ]:
array_of_img = read_directory("codebook_imgs/Depth_imgs")
print(type(array_of_img))
#print(array_of_img)
print(len(array_of_img))
print(array_of_img[0].shape)
print(array_of_img[1].shape)
print(array_of_img[2].shape)
print(array_of_img[3].shape)
print(array_of_img[4].shape)
print(array_of_img[5].shape)

<class 'list'>
2000
(111, 129)
(75, 167)
(104, 71)
(115, 80)
(184, 176)
(113, 80)


In [ ]:
################################  Threading  #####################################

In [ ]:
def HONV_feature(imgs,q):
    l = len(imgs)
    image_vectors = np.zeros((48))
    for i in range(l):
        img_feature = HONV.img_Vector(imgs[i]) #HONV
        #print(img_feature.shape)
        
        image_vectors = np.vstack((image_vectors,img_feature))
    image_vectors = np.delete(image_vectors, 0, axis=0)    
    q.put(image_vectors)

In [ ]:
#using threading for the acceleration of feature extration
def multithreading(array_of_img):
    thread_num = 8
    q =Queue() r
    threads = []
    data = array_of_img
    
    l = len(data)
    data_batch = int(l/thread_num)
    
    for i in range(thread_num):
        t = threading.Thread(target=HONV_feature,args=(data[data_batch*i: data_batch*(i+1)],q))
        t.start()
        threads.append(t)
    
    for thread in threads:
        thread.join()
    
    results = np.zeros((48))
    for i in range(thread_num):
        results = np.vstack((results,q.get()))
    results = np.delete(results, 0, axis=0)
    return results

In [ ]:
results = multithreading(array_of_img)
#print(results)
print(results.shape)
print(type(results))

(888400, 48)
<class 'numpy.ndarray'>


In [ ]:
################################  K-Means  #####################################

In [ ]:
#n_clusters means the number of cluster center, the size of cokebook
kmeans = KMeans(n_clusters=512, random_state=0, max_iter=300).fit(results)
print(kmeans.labels_)
centers = kmeans.cluster_centers_
print(centers)
print(centers.shape)

[167 167 153 ... 503 276  31]
[[1.60265497e-02 1.61064827e-03 2.29771700e-03 ... 5.47043666e-02
  1.95590880e-02 1.55841430e-03]
 [9.75308429e-01 1.66203021e-02 2.79286512e-02 ... 4.35710452e-03
  3.40102292e-03 1.17804450e-03]
 [1.02514379e-01 7.55340100e-04 1.02335680e-03 ... 3.56009289e-04
  5.27459844e-04 3.26268350e-03]
 ...
 [9.61650892e-01 6.99999707e-04 9.65551533e-04 ... 7.76895772e-04
  5.61889992e-04 5.39777850e-03]
 [7.29604034e-02 9.83290567e-03 1.78034075e-02 ... 2.67655317e-01
  1.29610601e-01 7.24086125e-03]
 [3.16185871e-02 1.50676877e-03 2.27234639e-03 ... 4.16018170e-04
  2.78426828e-04 2.63968636e-03]]
(512, 48)


In [ ]:
joblib.dump(kmeans,  'kmeans_HONV_512.pkl')

['kmeans_HONV_512.pkl']

In [ ]:
codebook = joblib.load('kmeans_HONV_512.pkl')